In [2]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
pd.options.display.max_rows = 120
import matplotlib as mpl
import sys
from matplotlib.pyplot import figure
figure(num=None, figsize=(8, 6), dpi=80, facecolor='w', edgecolor='k')
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import scipy

<Figure size 640x480 with 0 Axes>

In [3]:
dataset_material = pd.read_excel('Data_Ghiringhelli.xlsx', 'Material Data')
dataset_atomic = pd.read_excel('Data_Ghiringhelli.xlsx',   'Atomic Data')
dataset_atomic.columns = ['Z', 'A', 'IP', 'EA', 'HOMO', 'LUMO', 'rs', 'rp', 'rd', '1D']

In [4]:
def get_row(element,dataframe):
    return (dataframe.loc[dataframe['A']==element])

def collecting_descriptor(a,b, dataframe=dataset_atomic):
    row_a = get_row(element=a, dataframe=dataframe)
    row_b = get_row(element=b, dataframe=dataframe)
    return row_a,row_b

def cross_check(dataframe=dataset_material, y_array=dataset_material['DE'].values):
    feature_array = []
    for kk in range(len(dataframe.values)):
        el_a,el_b   = (dataframe.values)[kk][2:4]
        row_a,row_b = collecting_descriptor(a=el_a, b=el_b)
        res = formula(row_a, row_b)
        feature_array.append(res[0])
     
    feature_array = np.array(feature_array).reshape(-1,1)
    val1, val2 = scipy.stats.pearsonr(feature_array.flatten(), y_array)
    mse = []
    for ii in range(10):
        X_train, X_test, y_train, y_test = train_test_split(feature_array, y_array, test_size=0.1, random_state=ii)
        regressor = LinearRegression()
        regressor.fit((np.array(X_train)).reshape(-1,1), y_train)
        y_pred = regressor.predict((np.array(X_test)).reshape(-1,1))
        mse.append(mean_squared_error(y_test,y_pred))
    avg = float(np.average(mse))
    print('avg rmse = ',avg)
    print('pear_cof = ',np.abs(val1))
    print('p-value  = ',val2)

In [5]:
# Luca descriptor
def formula(a,b):
    return ((b['EA'].values-b['IP'].values)/(a['rp'].values**2))

In [6]:
#(rp_B - rd_A)/((rp_A**3))
def formula(a,b):    
    return ((b['rp'].values-a['rd'].values)/(a['rp'].values**3))

#(exp(rp_A) - sqrt(fabs(rp_B)))/((rp_A**3)) #low rmse for 18000 features
def formula(a,b):
    return (np.exp(a['rp'].values)-np.sqrt(np.abs(b['rp'].values)))/(a['rp'].values**3)
    

#(sqrt(fabs(HOMOKS_B)) - sqrt(fabs(LUMOKS_A)))/((rp_A**3))
def formula(a,b):    
    return (np.sqrt(np.abs(b['HOMO'].values)) - np.sqrt(np.abs(a['LUMO'].values)))/(a['rp'].values**3)


#high pearson coeff -  (exp(rp_A) - rp_B)/((rp_A**3))
def formula(a,b):
    return (np.exp(a['rp'].values)-b['rp'].values)/(a['rp'].values**3)

In [11]:
cross_check() #(sqrt(fabs(HOMOKS_B)) - sqrt(fabs(LUMOKS_A)))/((rp_A**3))

avg rmse =  0.02523561611917982
pear_cof =  0.9398192483687128
p-value  =  4.613719563062014e-39


In [11]:
cross_check() #(rp_B - rd_A)/((rp_A**3))

avg rmse =  0.024361084093093326
pear_cof =  0.9391075757445995
p-value  =  7.281535809764828e-39


In [7]:
cross_check() #luca descriptor

avg rmse =  0.021190747068464563
pear_cof =  0.9474855814299248
p-value  =  2.301317180384929e-41


In [30]:
cross_check() #18000 features low rmse

avg rmse =  0.017837393344104193
pear_cof =  0.9583208097660397
p-value  =  2.749045963173275e-45


In [7]:
cross_check() #18000 features highest pearson coeff

avg rmse =  0.01891242388583298
pear_cof =  0.9590672969687573
p-value  =  1.3536908279966107e-45


In [7]:
df = pd.read_csv('first_18000_files.csv')
del df['Unnamed: 0']
print('low rmse - ',(df[df['rmse'] == np.min(df['rmse'].values)])['formulas'].values[0])
print('high pearson coeff - ',(df[df['pearson_coeff'] == np.max(df['pearson_coeff'].values)])['formulas'].values[0])

low rmse -  (exp(rp_A) - sqrt(fabs(rp_B)))/((rp_A**3))
high pearson coeff -  (exp(rp_A) - rp_B)/((rp_A**3))
